In [1]:
import sys # funkcje systemowe, np. wyjscie z programu
import os # scieżki, funkcje systemowe itd.
import pyautogui # zewnetrzna biblioteka pip install pyautogui -> do zabawy z dźwiękiem
import tensorflow as tf # wgranie modelu sieci neuronowej
import numpy as np # operacje na macierzach
import cv2 # OpenCV2 -> computer vision


In [2]:
new_model = tf.keras.models.load_model("upvsdown2.model")

#parametry do cv2.putText()
font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 50)
fontScale = 1
color = (255, 0, 0)
thickness = 2

CATEGORIES = ["up", "down"]

In [3]:
import math


class EuclideanDistTracker:
    
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids



In [4]:
#funkcja która pobiera zdjęcie i zwraca predykcje 
def frame_to_Network(imgArr):
    imgArrNew = imgArr.reshape(-1, 130, 130, 1) 
    prediction = new_model.predict(imgArrNew)
    label = np.argmax(prediction)
    
    return CATEGORIES[int(prediction[0][0])]




In [6]:

# inicjalizacja trackera
tracker = EuclideanDistTracker()
cap = cv2.VideoCapture(0)
object_detector = cv2.createBackgroundSubtractorMOG2(history=70, varThreshold=100)

while (True):
    
    #pobieramy klatka po klatce, i wrzucamy je do funkcji frame_to_Network
    ret, frame = cap.read()
    display = frame.copy()
    height, width, _ = frame.shape
    
    #roi
    roi = frame
    # robimy maske background subtracker, tego co sie porusza,
    # aby tracker mogl to wykryc
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    
    #resizujemy klatke na 130x130, oraz nakladamy filtr czarno-biały
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resizedImg = cv2.resize(gray, (130,130))
    
    
    #wypisujemy na ekran co wykryła nasza sieć "up lub down"    
    pred_text = cv2.putText(frame, frame_to_Network(resizedImg), org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
    
    
    #sprawdzenie czy nasza sieć wykryła up lub down, jeśli tak
    #to przyciszamy albo pogłaśniamy dźwięk
    

    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area > 1000:
            #cv2.drawContours(roi, [cnt], -1, (0, 255, 0), 2)
            x, y, w, h = cv2.boundingRect(cnt)


            detections.append([x, y, w, h])

    # 2. Object Tracking
    boxes_ids = tracker.update(detections)
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)

        
    cv2.imshow("Mask", mask)
    cv2.imshow('frame', frame)

    #wychodzimy z pętli za pomocą klawisza "Q" - quit
    k = cv2.waitKey(10)
    if k==ord('q'):
            break
    if k == ord('s'):
            if frame_to_Network(resizedImg) == "up":
                pyautogui.press("volumeup")
            if frame_to_Network(resizedImg) == "down":
                pyautogui.press("volumedown")

# wyłącz przechwytywanie
cap.release()
cv2.destroyAllWindows()


{11: (294, 346), 12: (278, 246), 13: (323, 342)}
{13: (323, 342), 12: (288, 238), 14: (349, 338)}
{14: (355, 333), 12: (288, 238)}
{14: (355, 333), 12: (297, 240)}
{15: (338, 286)}
{15: (345, 289), 16: (198, 308)}
{16: (214, 291), 15: (345, 289), 17: (133, 408)}
{19: (89, 397), 20: (261, 279)}
{19: (89, 397), 20: (256, 285), 21: (31, 321)}
{19: (90, 392), 21: (31, 321), 20: (256, 285)}
{19: (90, 392), 21: (31, 301), 20: (256, 285)}
{19: (90, 392), 21: (31, 301), 20: (252, 290)}
{19: (90, 388), 21: (31, 301), 20: (252, 290)}
{19: (90, 388), 21: (26, 294), 20: (252, 290), 22: (381, 314)}
{19: (90, 388), 21: (26, 294), 20: (255, 285), 22: (381, 314)}
{19: (90, 387), 22: (381, 314), 21: (26, 294), 20: (255, 285)}
{19: (92, 366), 23: (390, 288), 24: (290, 264)}
{19: (92, 366), 23: (394, 299), 24: (290, 264)}
{19: (92, 366), 23: (394, 299), 24: (292, 262)}
{19: (92, 366), 23: (395, 301), 24: (292, 262)}
{19: (92, 366), 23: (395, 301), 24: (309, 257)}
{23: (398, 301), 24: (309, 257)}
{26: (31